In [1]:
import requests
from requests.auth import HTTPDigestAuth
import json

# Replace with the correct URL
url = "http://api_url"

# It is a good practice not to hardcode the credentials. So ask the user to enter credentials at runtime
myResponse = requests.get(url,auth=HTTPDigestAuth(raw_input("username: "), raw_input("Password: ")), verify=True)
#print (myResponse.status_code)

# For successful API call, response code will be 200 (OK)
if(myResponse.ok):

    # Loading the response data into a dict variable
    # json.loads takes in only binary or string variables so using content to fetch binary content
    # Loads (Load String) takes a Json file and converts into python data structure (dict or list, depending on JSON)
    jData = json.loads(myResponse.content)

    print("The response contains {0} properties".format(len(jData)))
    print("\n")
    for key in jData:
        print key + " : " + jData[key]
else:
  # If response code is not ok (200), print the resulting http error code with description
    myResponse.raise_for_status()

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-1-6d8a1a57867e>, line 23)

In [21]:
import requests
import json
import time
import hashlib
import pandas as pd

URL = "https://s5.securepilot.com/parking/v1/query_record"
api_key = "PARK-eMPpua7y17"
api_secret = "2PHIAGI5qv5IhMGZ56iy"


def getParkDataResponse(URL, api_key, api_secret):
    current_unix_time = str(int(time.time()))
    api_token = hashlib.sha1((api_secret + current_unix_time).encode('utf-8')).hexdigest()

    print("------------- RESTful API parameter --------------")
    print("api_key: ", api_key)
    print("api_secret: ", api_secret)
    print("unix_time: ", current_unix_time)
    print("api_token: ", api_token)
    print("--------------------------------------------------\n")

    payload = {'api_key': api_key, 'api_token': api_token, 'time': current_unix_time }
    return requests.post(URL, data=payload)

def getPraseTargetData(req):
    prase_data = req.json()
    prase_data = prase_data['value']['hits']

    prase_data_from_json = []
    # columes in data frame
    prase_data_columes = ['_id', '_index', 'create_at_time','recv_at_time','data','temperature',
                          'company','road','slot_num','gwip']
    # get data by list
    for each in prase_data:
        print([ each['_id'], each['_index'], each['_source']['create_at'], each['_source']['recv'], 
                each['_source']['data'], each['_source']['dataParse']['temperature'], 
                each['_source']['extra']['company'],  each['_source']['extra']['road'], each['_source']['extra']['slot_num'],
                each['_source']['gwip']
              ])
        prase_data_from_json.append([
                each['_id'], each['_index'], each['_source']['create_at'], each['_source']['recv'],
                each['_source']['data'], each['_source']['dataParse']['temperature'], 
                each['_source']['extra']['company'],  each['_source']['extra']['road'], each['_source']['extra']['slot_num'],
                each['_source']['gwip']
        ])
       
    return pd.DataFrame(prase_data_from_json, columns=prase_data_columes) 


# main funciton
req = getParkDataResponse(URL, api_key, api_secret)

if(req.ok):
    print("Request URL: ",req.url, " Status: OK")
    df = getPraseTargetData(req)
    df.to_csv("temp.csv")
else:
     print("Status: FAIL ",req.status_code)
    
# #     print(req.text)



------------- RESTful API parameter --------------
api_key:  PARK-eMPpua7y17
api_secret:  2PHIAGI5qv5IhMGZ56iy
unix_time:  1500884351
api_token:  61245707ff18911c6c84fe060dc3dd7736a636b7
--------------------------------------------------

Request URL:  https://s5.securepilot.com/parking/v1/query_record  Status: OK
['AV1zjj-Q-R03vmj0KhED', 'parking_record_index_v2', '2017-07-24T07:44:48.526Z', '2017-07-24T07:43:42.000Z', 'fc000105600bb00023', 35, 'yilan', '宜蘭市中山路二段', 'try', '192.168.0.225']
['AV1zjj-L-R03vmj0KhEC', 'parking_record_index_v2', '2017-07-24T07:44:48.522Z', '2017-07-24T07:44:48.000Z', 'fc000105600bb00023', 35, 'yilan', '宜蘭市中山路二段', 'try', '10.5.180.16']
['AV1zjj-B-R03vmj0KhEB', 'parking_record_index_v2', '2017-07-24T07:44:48.512Z', '2017-07-24T07:44:48.000Z', 'fc000105600bb00023', 35, 'yilan', '宜蘭市中山路二段', 'try', '192.168.0.3']
['AV1zjj9q-R03vmj0KhEA', 'parking_record_index_v2', '2017-07-24T07:44:48.488Z', '2017-07-24T07:44:48.000Z', 'fc000105600bb00023', 35, 'yilan', '宜蘭市中山路二